<h1>Code Agent Secure Code Execution wtih E2B Sandbox</h1>



<p style="background-color:#faa635; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Copy Right:</b> The contant is originated from a course by DeepLearning.AI called <b>"Building Code Agents with Hugging Face smolagents"</b> Some parts may varied due to personal cusomization for research and observation purposses.</p>

When it comes to an agent that can write and execute codes on your system automatically, you definitely have to consider malicious attacks.

Imagine the following: 

💣 The agent runs a command that deletes all files on your system.

💣 It initiates an infinite loop, quickly consuming all resources and crashing your machine.

💣 It installs conflicting libraries, breaking dependencies or corrupting your environment.

Sounds scary, ain't it. But the good news is that we can secure our code execution to prevent malicious attacks. The follow shows some of the methods that we are going to see in this notebook as well:

<h2>🛡Built-In Safeguards</h2>

✅<b> Ignore undefined or unsafe commands</b>
 Any command not explicitly supported or allowed should be skipped automatically and return an error to show it was identified a “harmful command”.
 
✅<b> Secure import control</b>
 Only permit imports from a vetted list of libraries.
 
✅<b> Prevent infinite loops<b>
 Use loop counters, timeouts, or resource limits to detect and stop never-ending loops.

## Show behavior of custom python interpreter

In [ ]:
#!pip install git+https://github.com/huggingface/smolagents.git

In [1]:
from smolagents.local_python_executor import LocalPythonExecutor

custom_executor = LocalPythonExecutor(["numpy"])

In [2]:
def run_capture_exception(command: str):
    try:
        custom_executor(harmful_command)
    except Exception as e:
        print("ERROR:\n", e)

In [3]:
# Example 1: non-defined command
# In Jupyter it works
!echo Bad command

Bad command


In [4]:
# In our interpreter, it does not.
harmful_command="!echo Bad command"
run_capture_exception(harmful_command)

ERROR:
 Code parsing failed on line 1 due to: SyntaxError
!echo Bad command
 ^
Error: invalid syntax (<unknown>, line 1)


In [5]:
[
    're',
    'queue',
    'random',
    'statistics',
    'unicodedata',
    'itertools',
    'math',
    'stat',
    'time',
    'datetime',
    'collections',
    'numpy'
]

['re',
 'queue',
 'random',
 'statistics',
 'unicodedata',
 'itertools',
 'math',
 'stat',
 'time',
 'datetime',
 'collections',
 'numpy']

In [6]:
# Example 2: os not imported
harmful_command="""
import os
exit_code = os.system("echo Bad command")
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'import os' due to: InterpreterError: Import of os is not allowed. Authorized imports are: ['stat', 'math', 're', 'unicodedata', 'queue', 'itertools', 'statistics', 'random', 'collections', 'datetime', 'time', 'numpy']


In [7]:
# Example 3: random._os.system not imported
harmful_command="""
import random
random._os.system('echo Bad command')
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'random._os.system('echo Bad command')' due to: InterpreterError: Forbidden access to module: os


In [8]:
# Example 4: infinite loop
harmful_command="""
while True:
    pass
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'while True:
    pass' due to: InterpreterError: Maximum number of 1000000 iterations in While loop exceeded


In [ ]:
custom_executor = LocalPythonExecutor(["PIL"])

harmful_command="""
from PIL import Image

img = Image.new('RGB', (100, 100), color='blue')

i=0
while i < 10000:
    img.save('simple_image_{i}.png')
    i += 1
"""
# custom_executor(harmful_command)
# Let's not execute this but it would not error out, and it would bloat your system with images.

## Running in a sandbox

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

E2B_API_KEY = os.getenv("E2B_API_KEY")

In [10]:
from smolagents import CodeAgent, HfApiModel, Tool

model = HfApiModel()

class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def __init__(self, max_output_length: int = 40000):
        super().__init__()
        self.max_output_length = max_output_length

    def forward(self, url: str) -> str:
        try:
            import re

            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException

            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, self.max_output_length)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=model,
    executor_type="e2b",
    executor_kwargs={"api_key": E2B_API_KEY},
    max_steps=5
)

Initializing executor, hold on...

Collecting smolagents

  Downloading smolagents-1.14.0-py3-none-any.whl (114 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/114.5 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m114.5/114.5 kB[0m [31m4.7 MB/s[0m eta [36m0:00:00[0m
[?25h
Requirement already satisfied: jinja2>=3.1.4 in /usr/local/lib/python3.10/site-packages (from smolagents) (3.1.6)

Collecting rich>=13.9.4

  Downloading rich-14.0.0-py3-none-any.whl (243 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/243.2 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m243.2/243.2 kB[0m [31m15.3 MB/s[0m eta [36m0:00:00[0m
[?25h
Collecting python-dotenv
  Downloading python_dotenv-1.1.0-py3-none-any.whl (20 kB)

Collecting requests>=2.32.3
  Downloading requests-2.32.3-py3-none-any.whl (64 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/64.9 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m64.9/64.9 kB[0m [31m9.3 MB/s[0m eta [36m0:00:00[0m
[?25h
Collecting markdownify>=0.14.1
  Downloading markdownify-1.1.0-py3-none-any.whl (13 kB)

Collecting pillow>=11.0.0
  Downloading pillow-11.2.1-cp310-cp310-manylinux_2_28_x86_64.whl (4.6 MB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/4.6 MB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [91m━━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.7/4.6 MB[0m [31m22.2 MB/s[0m eta
[36m0:00:01[0m[2K     [91m━━━━━━━━━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m1.5/4.6 MB[0m [31m21.8
MB/s[0m eta [36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━━━[0m [32m2.6/4.6 MB[0m [31m24.3 MB/s[0m eta
[36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m[90m━━━━━━[0m [32m3.9/4.6 MB[0m [31m27.6 MB/s[0m eta
[36m0:00:01[0m[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m [32m4.6/4.6 MB[0m [31m29.3 MB/s[0m eta
[36m0:00:01[0m
[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m4.6/4.6 MB[0m [31m24.6 MB/s[0m eta [36m0:00:00[0m
[?25h
Collecting duckduckgo-search>=6.3.7
  Downloading duckduckgo_search-8.0.1-py3-none-any.whl (18 kB)

Collecting huggingface-hub>=0.28.0

  Downloading huggingface_hub-0.31.0-py3-none-any.whl (484 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/484.3 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m484.3/484.3 kB[0m [31m31.5 MB/s[0m eta [36m0:00:00[0m
[?25hRequirement already satisfied: click>=8.1.8 in /usr/local/lib/python3.10/site-packages (from 
duckduckgo-search>=6.3.7->smolagents) (8.1.8)
Requirement already satisfied: lxml>=5.3.0 in /usr/local/lib/python3.10/site-packages (from 
duckduckgo-search>=6.3.7->smolagents) (5.4.0)

Collecting primp>=0.15.0
  Downloading primp-0.15.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/3.3 MB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m[90m━━━━━━━━━━━━━━━━━[0m [32m1.9/3.3 MB[0m [31m60.6 MB/s[0m eta
[36m0:00:01[0m[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m3.3/3.3 MB[0m [31m47.4 MB/s[0m eta
[36m0:00:00[0m
[?25h
Requirement already satisfied: pyyaml>=5.1 in /usr/local/lib/python3.10/site-packages (from 
huggingface-hub>=0.28.0->smolagents) (6.0.2)

Collecting fsspec>=2023.5.0
  Downloading fsspec-2025.3.2-py3-none-any.whl (194 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/194.4 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m194.4/194.4 kB[0m [31m43.7 MB/s[0m eta [36m0:00:00[0m
[?25hRequirement already satisfied: tqdm>=4.42.1 in /usr/local/lib/python3.10/si

E2B is running

<div style="font-size: 14px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> The agent is not successful in the following run due to 403 Permission restrictions.
</div>


In [ ]:
output = agent.run(
    "Give me one of the top github repos from organization huggingface."
)
print("E2B executor result:", output)